## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP1 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [42]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [43]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_c.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_c.csv", sep=";")
display(df_tarjetas.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [24]:
print(df_tarjetas.columns)  # Muestra las columnas disponibles

Index(['id_cliente', 'antiguedad_cliente', 'estado_civil', 'estado_cliente',
       'gastos_ult_12m', 'genero', 'limite_credito_tc', 'nivel_educativo',
       'nivel_tarjeta', 'operaciones_ult_12m', 'personas_a_cargo'],
      dtype='object')


In [25]:
# Se establece qué columnas se eliminan
col_eliminar_creditos = []  # No hay columnas definidas para eliminar
col_eliminar_tarjetas = ['nivel_tarjeta']

# Verificar qué columnas existen antes de eliminarlas
col_existentes_creditos = [col for col in col_eliminar_creditos if col in df_creditos.columns]
col_existentes_tarjetas = [col for col in col_eliminar_tarjetas if col in df_tarjetas.columns]

# Se ejecuta la operación eliminando solo las columnas existentes
df_creditos.drop(col_existentes_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_existentes_tarjetas, inplace=True, axis=1)

# Mostrar las primeras filas para verificar cambios
print("Primeras filas de df_creditos:")
print(df_creditos.head(5))

print("Primeras filas de df_tarjetas:")
print(df_tarjetas.head(5))

# Imprimir el número de filas antes de aplicar filtros
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

# Validar si la columna 'edad' existe antes de aplicar el filtro
if 'edad' in df_creditos.columns:
    temp = df_creditos[df_creditos['edad'] < 90]  # Filtrar por edad
else:
    print("Advertencia: La columna 'edad' no existe en df_creditos.")
    temp = df_creditos.copy()  # No aplicar filtro si la columna no existe

# Validar si la columna 'regla_pct_ingresos' existe antes de aplicar el filtro
if 'regla_pct_ingresos' in temp.columns:
    temp_c = temp[temp['regla_pct_ingresos'] == 'ok']  # Filtrar por regla de negocio
else:
    print("Advertencia: La columna 'regla_pct_ingresos' no existe en df_creditos.")
    temp_c = temp.copy()  # No aplicar filtro si la columna no existe

# Imprimir el número de filas después de aplicar filtros
print(f"Filas después del filtro: {temp_c.shape[0]}")


Primeras filas de df_creditos:
    id_cliente  edad  importe_solicitado  duracion_credito  \
0  713061558.0    22               35000                 3   
1  768805383.0    21                1000                 2   
2  818770008.0    25                5500                 3   
3  713982108.0    23               35000                 2   
4  710821833.0    24               35000                 4   

   antiguedad_empleado situacion_vivienda  ingresos objetivo_credito  \
0                123.0           ALQUILER     59000         PERSONAL   
1                  5.0             PROPIA      9600        EDUCACIÓN   
2                  1.0           HIPOTECA      9600            SALUD   
3                  4.0           ALQUILER     65500            SALUD   
4                  8.0           ALQUILER     54400            SALUD   

   pct_ingreso  tasa_interes  estado_credito falta_pago  
0         0.59         16.02               1          Y  
1         0.10         11.14               0   

In [26]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,713061558.0,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,24.0,2.0


Limpieza de los datos (filtrado a nivel de filas)

In [7]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)  

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,EDUCACIÓN,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,SALUD,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,SALUD,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,SALUD,0.55,14.27,1,Y,ok


In [8]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90] # 4

# Otro filtro posible: por la regla de negocio agregada

temp_c = temp[temp['regla_pct_ingresos'] == 'ok'] # 15

print(f"Filas después del filtro: {temp_c.shape[0]}")

Filas antes del filtro: 10127
Filas después del filtro: 10108


Filtros de filas generados por la estudiante - CRÉDITOS:

In [9]:
print(df_creditos['duracion_credito'].describe())  # Estadísticas generales
print(df_creditos['duracion_credito'].unique())   # Valores únicos
print(df_creditos.dtypes)  # Tipos de datos

count    10127.000000
mean         2.995556
std          0.817230
min          2.000000
25%          2.000000
50%          3.000000
75%          4.000000
max          4.000000
Name: duracion_credito, dtype: float64
[3 2 4]
id_cliente             float64
edad                     int64
importe_solicitado       int64
duracion_credito         int64
antiguedad_empleado    float64
situacion_vivienda      object
ingresos                 int64
objetivo_credito        object
pct_ingreso            float64
tasa_interes           float64
estado_credito           int64
falta_pago              object
regla_pct_ingresos      object
dtype: object


In [10]:
# Filtro 1: Duración del crédito
df_creditos = df_creditos[df_creditos['duracion_credito'] >= 3]
print(f"Filas después del filtro de duración del crédito: {df_creditos.shape[0]}")
print(df_creditos.head())  # Tipos de datos

Filas después del filtro de duración del crédito: 6723
    id_cliente  edad  importe_solicitado  duracion_credito  \
0  713061558.0    22               35000                 3   
2  818770008.0    25                5500                 3   
4  710821833.0    24               35000                 4   
6  709106358.0    26               35000                 3   
7  810347208.0    24               35000                 4   

   antiguedad_empleado situacion_vivienda  ingresos objetivo_credito  \
0                123.0           ALQUILER     59000         PERSONAL   
2                  1.0           HIPOTECA      9600            SALUD   
4                  8.0           ALQUILER     54400            SALUD   
6                  8.0           ALQUILER     77100        EDUCACIÓN   
7                  5.0           ALQUILER     78956            SALUD   

   pct_ingreso  tasa_interes  estado_credito falta_pago regla_pct_ingresos  
0         0.59         16.02               1          Y       

Filtros de filas generados por la estudiante - TARJETAS:

In [11]:
# 1. Filtro por limite_credito_tc (Eliminar tarjetas con límite menor a 5,000)
print(df_tarjetas['limite_credito_tc'].describe())  # Estadísticas generales
print(df_tarjetas['limite_credito_tc'].unique())   # Valores únicos
print(df_tarjetas.dtypes)  # Tipos de datos

count    10127.000000
mean      8631.953698
std       9088.776650
min       1438.300000
25%       2555.000000
50%       4549.000000
75%      11067.500000
max      34516.000000
Name: limite_credito_tc, dtype: float64
[ 4010. 12691.  8256. ...  5409. 10388. 14657.]
id_cliente             float64
antiguedad_cliente     float64
estado_civil            object
estado_cliente          object
gastos_ult_12m         float64
genero                  object
limite_credito_tc      float64
nivel_educativo         object
operaciones_ult_12m    float64
personas_a_cargo       float64
dtype: object


In [12]:
df_tarjetas = df_tarjetas[df_tarjetas['limite_credito_tc'] >= 3000]
print(f"Filas después del filtro de límite de crédito: {df_tarjetas.shape[0]}")

Filas después del filtro de límite de crédito: 6688


In [13]:
# 2. Filtro por estado_cliente (Eliminar tarjetas de clientes inactivos)
print(df_tarjetas['estado_cliente'].unique())  # Verifica los estados posibles
print(df_tarjetas['estado_cliente'].value_counts())  # Ver distribución de valores
print(df_tarjetas.dtypes)  # Verificar tipo de dato



['ACTIVO' 'PASIVO']
estado_cliente
ACTIVO    5667
PASIVO    1021
Name: count, dtype: int64
id_cliente             float64
antiguedad_cliente     float64
estado_civil            object
estado_cliente          object
gastos_ult_12m         float64
genero                  object
limite_credito_tc      float64
nivel_educativo         object
operaciones_ult_12m    float64
personas_a_cargo       float64
dtype: object


In [14]:
df_tarjetas = df_tarjetas[df_tarjetas['estado_cliente'] == 'ACTIVO']
print(f"Filas después del filtro de estado del cliente: {df_tarjetas.shape[0]}")

Filas después del filtro de estado del cliente: 5667


In [15]:
# 3. Filtro por operaciones_ult_12m (Eliminar tarjetas con menos de 10 operaciones en el último año)
print(df_tarjetas['operaciones_ult_12m'].describe())  # Estadísticas generales
print(df_tarjetas['operaciones_ult_12m'].unique())   # Valores únicos
print(df_tarjetas.dtypes)  # Tipos de datos


count    5667.000000
mean       69.039351
std        25.027137
min        11.000000
25%        51.000000
50%        70.000000
75%        84.000000
max       139.000000
Name: operaciones_ult_12m, dtype: float64
[ 24.  42.  33.  20.  26.  28.  31.  36.  32.  17.  27.  21.  30.  18.
  22.  23.  40.  43.  38.  37.  35.  29.  25.  41.  15.  57.  14.  34.
  19.  47.  13.  16.  39.  50.  52.  48.  49.  44.  55.  53.  11.  45.
  54.  51.  63.  46.  58.  59.  60.  61.  78.  64.  65.  67.  56.  71.
  75.  69.  76.  74.  62.  66.  84.  82.  88.  68.  86.  72.  79.  80.
  73.  85.  70.  83.  91.  87.  77.  81. 103.  93.  96.  92.  89.  90.
  94. 100. 102.  98.  95.  97. 101. 104. 105. 106.  99. 107. 118. 108.
 109. 122. 113. 112. 127. 114. 124. 110. 120. 125. 121. 117. 111. 126.
 134. 116. 119. 129. 131. 115. 128. 139. 123. 130. 138. 132.]
id_cliente             float64
antiguedad_cliente     float64
estado_civil            object
estado_cliente          object
gastos_ult_12m         float64
gener

In [16]:
df_tarjetas = df_tarjetas[df_tarjetas['operaciones_ult_12m'] >= 50]
print(f"Filas después del filtro de operaciones en el último año: {df_tarjetas.shape[0]}")


Filas después del filtro de operaciones en el último año: 4295


Integración de datos

In [44]:
df_integrado = pd.merge(temp_c, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizados: {coincidencias}")

Filas del dataset integrado con los filtros realizados: 10123


In [45]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 22


In [46]:
df_integrado.head(1)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,...,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


##### Modificaciones para le proceso de clusterización
Se resguardan los datos apra aplicar otras transformaciones rewqueridas por las técnicas.

In [47]:
df_integrado_c = df_integrado.copy()
df_integrado_c.head(1)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,...,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


#### Transformación de atributos

Atributos nominales que se modifican los valores

In [71]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_credito
cambios_estado_credito = {
    0: 'P',
    1 : 'C',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')

# Sobre este resultado será necesario eliminar las columnas auxiliares

Atributos numéricos que se discretizan

In [72]:
df_integrado['antiguedad_empleado'].describe()

count    4143.000000
mean        3.989138
std         2.851826
min         0.000000
25%         2.000000
50%         4.000000
75%         6.000000
max        11.000000
Name: antiguedad_empleado, dtype: float64

In [73]:
# Antiguedad del empleado
# Asignamos etiquetas para pasarlos a una forma nominal

etiquetas_a_e = ['menor_5', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleados_N = pd.cut(df_integrado['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

antiguedad_empleados_N.value_counts()

# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(df_integrado['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

tasa_interes_N.value_counts()

# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_integrado['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()

operaciones_ult_12m
50_a_75      1915
75_a_100     1705
mayor_100     671
menor_15        0
15_a_30         0
30_a_50         0
Name: count, dtype: int64

In [74]:
df_integrado.columns

Index(['id_cliente', 'edad', 'importe_solicitado', 'duracion_credito',
       'antiguedad_empleado', 'situacion_vivienda', 'ingresos',
       'objetivo_credito', 'pct_ingreso', 'tasa_interes', 'estado_credito',
       'falta_pago', 'regla_pct_ingresos', 'antiguedad_cliente',
       'estado_civil', 'estado_cliente', 'gastos_ult_12m', 'genero',
       'limite_credito_tc', 'nivel_educativo', 'operaciones_ult_12m',
       'personas_a_cargo'],
      dtype='object')

In [75]:
print(df_integrado.columns)


Index(['id_cliente', 'edad', 'importe_solicitado', 'duracion_credito',
       'antiguedad_empleado', 'situacion_vivienda', 'ingresos',
       'objetivo_credito', 'pct_ingreso', 'tasa_interes', 'estado_credito',
       'falta_pago', 'regla_pct_ingresos', 'antiguedad_cliente',
       'estado_civil', 'estado_cliente', 'gastos_ult_12m', 'genero',
       'limite_credito_tc', 'nivel_educativo', 'operaciones_ult_12m',
       'personas_a_cargo'],
      dtype='object')


In [76]:
# Agregar en este listado otros atributos que pudieran discretizarse o transformarse
# Lista de columnas a eliminar
col_eliminar_final = [
    'estado_civil', 'estado_credito', 'antiguedad_empleado', 'edad',
    'antiguedad_cliente', 'ingresos', 'pct_ingreso', 'tasa_interes',
    'regla_pct_ingresos', 'gastos_ult_12m', 'limite_credito_tc',
    'operaciones_ult_12m', 'id_cliente'
]

# Filtrar solo las columnas que existen en df_integrado
col_existentes = [col for col in col_eliminar_final if col in df_integrado.columns]

# Eliminar las columnas existentes
df_integrado.drop(col_existentes, inplace=True, axis=1)

# Concatenar los DataFrames
df_final = pd.concat([
    operaciones_ult_12m_N, gastos_ult_12m_N, limite_credito_tc_N,
    antiguedad_cliente_N, tasa_interes_N, ingresos_N, pct_ingreso_N,
    antiguedad_empleados_N, edad_N, estado_civil_N, estado_credito_N,
    df_integrado
], axis=1)

# Mostrar las primeras filas del DataFrame final
df_final.head(5)



,operaciones_ult_12m,gastos_ult_12m,limite_credito_tc,antiguedad_cliente,tasa_interes,ingresos,pct_ingreso,antiguedad_empleado,edad,estado_civil_N,estado_credito_N,importe_solicitado,duracion_credito,situacion_vivienda,objetivo_credito,falta_pago,estado_cliente,genero,nivel_educativo,personas_a_cargo
0,50_a_75,2k_a_4k,5k_a_10k,2y_a_4y,7p_a_15p,hasta_20k,20_a_40,menor_5,menor_25,S,C,4200,4,PROPIA,PERSONAL,Y,ACTIVO,F,UNIVERSITARIO_INCOMPLETO,3.0
1,50_a_75,2k_a_4k,mayor_10k,2y_a_4y,15p_a_20p,mayor_100k,hasta_20,5_a_10,25_a_30,C,P,18000,3,ALQUILER,EDUCACIÓN,N,ACTIVO,M,UNIVERSITARIO_COMPLETO,3.0
2,50_a_75,2k_a_4k,mayor_10k,2y_a_4y,15p_a_20p,mayor_100k,hasta_20,mayor_10,25_a_30,C,C,20000,2,HIPOTECA,INVERSIONES,N,ACTIVO,M,POSGRADO_COMPLETO,3.0
3,50_a_75,2k_a_4k,mayor_10k,2y_a_4y,7p_a_15p,mayor_100k,hasta_20,5_a_10,menor_25,D,P,2500,2,PROPIA,EDUCACIÓN,N,ACTIVO,M,POSGRADO_INCOMPLETO,3.0
4,50_a_75,2k_a_4k,5k_a_10k,mayor_4y,7p_a_15p,mayor_100k,hasta_20,5_a_10,25_a_30,C,P,25000,3,ALQUILER,PERSONAL,N,ACTIVO,M,UNIVERSITARIO_COMPLETO,0.0


Exportación de metadatos

In [77]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df_final, title="Reporte de preparación de datos - Dataset integrado")
profile.to_file("../../docs/reporte_datos_integrados.html")

Export report to file: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [78]:
print(f"Cantidad de columnas del dataset final: {df_final.shape[1]}")
print(f"Cantidad de filas del dataset final: {df_final.shape[0]}")

Cantidad de columnas del dataset final: 20
Cantidad de filas del dataset final: 4291


Exportación del dataset

In [79]:
df_final.to_csv("../../data/final/datos_finales.csv", sep=';', index=False)

#### Ajustes para el proceso de clusterización

Se realizan las transformaciones propias del proceso de descubrimiento de grupos

1. Eliminación de valores nulos

In [48]:
# Filtrado de valores nulos
df_filtrado = df_integrado_c[(df_integrado_c['tasa_interes'].notnull()) & (df_integrado_c['antiguedad_empleado'].notnull())]
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8899 entries, 0 to 10122
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_cliente           8899 non-null   float64
 1   edad                 8899 non-null   int64  
 2   importe_solicitado   8899 non-null   int64  
 3   duracion_credito     8899 non-null   int64  
 4   antiguedad_empleado  8899 non-null   float64
 5   situacion_vivienda   8899 non-null   object 
 6   ingresos             8899 non-null   int64  
 7   objetivo_credito     8899 non-null   object 
 8   pct_ingreso          8899 non-null   float64
 9   tasa_interes         8899 non-null   float64
 10  estado_credito       8899 non-null   int64  
 11  falta_pago           8899 non-null   object 
 12  antiguedad_cliente   8899 non-null   float64
 13  estado_civil         8899 non-null   object 
 14  estado_cliente       8899 non-null   object 
 15  gastos_ult_12m       8899 non-null   float

In [52]:
eliminar = ['regla_pct_ingresos']
df_filtrado.drop(columns=eliminar, inplace=True, errors='ignore') 
df_filtrado.head(1)


C:\Users\mdlid\AppData\Local\Temp\ipykernel_24572\2708100472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.drop(columns=eliminar, inplace=True, errors='ignore')


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,...,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0


2. Procesamiento de atributos nominales
Se realiza una binarización para aquellos métodos que sean incompatibles con datos no numéricos.

In [53]:
data = pd.get_dummies(df_filtrado)
data.head(1)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,ingresos,pct_ingreso,tasa_interes,estado_credito,antiguedad_cliente,...,nivel_educativo_DESCONOCIDO,nivel_educativo_POSGRADO_COMPLETO,nivel_educativo_POSGRADO_INCOMPLETO,nivel_educativo_SECUNDARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_COMPLETO,nivel_educativo_UNIVERSITARIO_INCOMPLETO,nivel_tarjeta_Blue,nivel_tarjeta_Gold,nivel_tarjeta_Platinum,nivel_tarjeta_Silver
0,713061558.0,22,35000,3,123.0,59000,0.59,16.02,1,36.0,...,False,False,False,False,True,False,True,False,False,False


Se vuelve a generar un reporte para observar los metadatos:

In [55]:
from ydata_profiling import ProfileReport

profile = ProfileReport(data, title="Reporte de preparación de datos - Dataset para clusterización")
profile.to_file("../../docs/reporte_datos_integrados_clusterizacion.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 30.28it/s]


In [56]:
print(f"Cantidad de columnas del dataset final - clusterización: {data.shape[1]}")
print(f"Cantidad de filas del dataset final - clusterización: {data.shape[0]}")

Cantidad de columnas del dataset final - clusterización: 44
Cantidad de filas del dataset final - clusterización: 8899


In [57]:
data.to_csv("../../data/final/datos_finales_clusterizacion.csv", sep=';', index=False)